# Demystifying Decorators

## Some you may have seen
- @staticmethod
- @classmethod
- @property
- @app.route in flask or similiar in other frameworks


## What's a decorator?
It's an @ symbol followed by some name sitting just above a function, method, or class definition.

## Well, yeah... but what is it *really*?
The whole thing has three parts:
- @ symbol
- some name
- function, method, or class definition

The key is that @ symbol. It's an operator just like +, -, *, /, etc., and it has two operands:

- A callable
- A function or class definition

## What's it do with them?

It is syntactic sugar for the following pattern:

In [1]:
# say we have some callable thing:

def times_two(n):
    return 2 * n

In [2]:
# and some other callable thing:

def my_decorator(obj):
    print "Wrapping %s!" % obj.__name__
    return obj

In [3]:
# and we pass the first thing into the second one and assign the result back to the first one.

times_two = my_decorator(times_two)

Wrapping times_two!


**The @ operator shortens this pattern. It invokes its first operand against its second operand, and it assigns the result back to the symbol for the second operand.**

## And that's it. You now know everything about decorators.

In [4]:
@my_decorator
def times_three(n):
    return 3 * n

Wrapping times_three!


In [5]:
times_three

<function __main__.times_three>

Remember, the first operand to @ is a callable, and **the operator takes care of invoking it**.

### Didn't you say it worked on class definitions?

In [6]:
@my_decorator
class B(object):
    pass

Wrapping B!


In [7]:
B

__main__.B

### And you're sure there's no  magic...?

In [8]:
def nope_youre_four(obj):
    return 4

In [9]:
@nope_youre_four
def im_a_function_that_returns_forty_two():
    return 42

In [10]:
im_a_function_that_returns_forty_two

4

In [11]:
type(im_a_function_that_returns_forty_two)

int

# Great! So what can we do with them?
A lot of cool, kind of sophisticated things that all depend on other powerful features of python.
## What features?
- Functions and classes in python are first class citizens. They can be
    - Passed as arguments
    - Returned as values
    - Created in any environment (in function invocations, class definitions, etc.)
- Closures
- Callable class instances (instances created from classes that behave like functions)
- Splats (\*args and \*\*kwargs)

### Passed as arguments

In [12]:
def square(x):
    return x * x

In [13]:
def mymap(func, args):
    return [func(a) for a in args]

In [14]:
mymap(square, range(5))

[0, 1, 4, 9, 16]

In [15]:
class A(object):
    def __init__(self, obj):
        self.obj = obj
        
    def __repr__(self):
        return "A(%s)" % str(self.obj)

In [16]:
mymap(A, range(5))

[A(0), A(1), A(2), A(3), A(4)]

### Inner definition, return value, and closure
When a function definition is encountered, a callable object is created that contains pointers to the code of the definiton and the current local environment. It's this callable object that is given a name. When a callable object is invoked, a new environment is created that has the original environment of definition as its parent, and the body of the callable is evaluated in that new environment.

In [17]:
def multiplier(n):
    def inner(x):
        return x * n
    return inner

In [18]:
mul_four = multiplier(4)

# mul_four is a callable object created when the inner definition was evaluated. The inner definition created
# a callable object that points to the code of its function body and its current environment. Its current
# environment is the one created for the invocation of multiplier. This means the callable object created
# when the definition for inner was evaluated can see whatever value was bound to n. And we can return this
# callable object as the result of invoking multiplier.

mul_four(3)

12

In [19]:
mul_four(range(4))

[0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]

### Objects can be callables, too!

In [20]:
class SquareIt(object):
    def __call__(self, x):
        return x * x
    
s = SquareIt()
s(3)

9

In [21]:
mymap(s, range(5))

[0, 1, 4, 9, 16]

### Splat Args

In [22]:
def add(a, b):
    return a + b

In [23]:
add(2, 3)

5

In [24]:
args = [2, 3]
add(*args)

5

# Back to decorators!

### How long does a function call take?

In [25]:
import functools
import time
from examples.util import get_name

def timeit(func):
    # @functools.wraps(func)
    def inner(*args, **kwargs):
        """This is adding some enhancement."""
        start = time.time()
        try:
            return func(*args, **kwargs)
        finally:
            duration = (time.time() - start)
            print "%s took %s" % (get_name(func), duration)

    # note that we return inner instead of func!
    return inner

In [26]:
import time

@timeit
def sum_to_n(a):
    """Sums integers."""
    total = a
    for x in range(a):
        total += x
        time.sleep(1)
    return total

In [27]:
sum_to_n(4)

__main__.sum_to_n took 4.00516200066


10

In [28]:
sum_to_n

<function __main__.inner>

In [29]:
sum_to_n.__doc__

'This is adding some enhancement.'

## Cool... but what about decorators that take arguments?

Okay, so you noticed that our decorators haven't had any parens at the end. That's because the @ operator takes care of invocation for us. Remember, the operator takes two operands: a callable as the first and some function, method, or class definition to apply it against as the second.

Since the @ operator will call the first operand for us, it doesn't seem like we can give it any arguments. What can we do? Let's take a clue from timeit.

In [30]:
# Say we want a decorator that prints a custom message for each thing it decorates.

# Since a global msg won't work, and we can't pass one into the decorator, let's make a factory function where
# msg is locally defined and create a function similar in structure to timeit inside that scope. Like with timeit,
# we can return a locally defined function as the value of an enclosing function.

def make_noisy(msg):
    def decorator(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            print msg
            return func(*args, **kwargs)
        return inner
    return decorator

# We have to get an instance of the decorator back out of that whole thing, so let's ask the factory for one:
noise_maker = make_noisy("Calling times_four!")

# noise_maker is a procedure constructed from the inner function definition and the environment where it was
# evaluated. I.e., it's a callable object that contains an environment where msg is bound to the string we passed
# into make_noisy. We can use it as the first operand of @.

@noise_maker
def times_four(n):
    return 4 * n

times_four(4)

Calling times_four!


16

### Can we skip that assignment step?

Sure, let's just call make_noisy right after the @. We don't have to give the callable we're getting out of it a name first.

In [31]:
@make_noisy("Calling times_five!")
def times_five(n):
    return 5 * n

times_five(5)

Calling times_five!


25

## Neat! Show me something practical I can do with it.

### Configurable Automatic Retries

In [32]:
import functools

def retry(num_retries=3):
    def decorator(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            exception = None
            attempts = num_retries + 1
            for attempt in xrange(1, attempts + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as ex:
                    print "Attempt %s failed!" % attempt
                    exception = ex
            raise exception
        return inner
    return decorator

In [33]:
import random

@retry(num_retries=5)
def flaky():
    if random.random() < 0.5:
        raise Exception("Boom!")
    return "Success!"

In [34]:
flaky()

'Success!'

### Memoization

We could have a function automatically remember its return values for given arguments. This can be useful if the function is computationally expensive or backed by a call that takes a long time to complete. Say it's reading from disk or making a network call.

In [35]:
# Here's a long running call to a service. The results don't change for the same arguments,
# and we don't need to remember every result.

# Terribad implementation for calculating fibonacci numbers for a given index. (Big-O complexity of 2^n)
def fib(n):
    if n < 2:
        return 1
    return fib(n - 1) + fib(n - 2)

### Let's keep the results from the last few calls around in case they're needed again soon.

In [36]:
import functools
from collections import deque

from examples.util import naive_hash


def memoizer(cache_size=5):
    # local state
    cache = {}
    args_queue = deque()
    cache_size = cache_size

    # inner functions
    def check_evict():
        if len(args_queue) > cache_size:
            old_key = args_queue.popleft()
            del cache[old_key]
            
    def should_compute(key):
        return key not in cache

    def memoize(key, value):
        cache[key] = value
        args_queue.append(key)
        
    def value(key):
        return cache[key]

    # the function that @ will call for us
    def decorator(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            key = naive_hash(args, kwargs)
            if should_compute(key):
                memoize(key, func(*args, **kwargs))
            check_evict()
            return value(key)
        return inner

    return decorator

In [37]:
@memoizer(cache_size=3)
def memfib(n):
    if n < 2:
        return 1
    return memfib(n - 1) + memfib(n - 2)

In [38]:
# This wouldn't complete in our lifetimes without memoization.
memfib(100)

573147844013817084101L

## Awesome, but wouldn't that be hard to test or change eviction policy?
Yep, and we already have a thing to capture local state and related functions: let's use a class.

In [39]:
import functools
from collections import deque

from examples.util import naive_hash


class Memoizer(object):
    def __init__(self, cache_size=5):
        self.cache = {}
        self.args_queue = deque()
        self.cache_size = cache_size

    def _check_evict(self):
        if len(self.args_queue) > self.cache_size:
            old_key = self.args_queue.popleft()
            del self.cache[old_key]
            
    def _should_compute(self, key):
        return key not in self.cache

    def _memoize(self, key, value):
        self.cache[key] = value
        self.args_queue.append(key)
        
    def _value(self, key):
        return self.cache[key]
        
    def __call__(self, func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            key = naive_hash(args, kwargs)
            if self._should_compute(key):
                self._memoize(key, func(*args, **kwargs))
            self._check_evict()
            return self._value(key)
        return inner

In [40]:
@Memoizer(cache_size=3)
def memfib2(n):
    if n < 2:
        return 1
    return memfib2(n - 1) + memfib2(n - 2)

In [41]:
memfib2(100)

573147844013817084101L

# Any Questions?